## Juntar y limpiar archivos

En este notebook vamos a limpiar y a generar groupbys en los archivos históricos para poder alimentar a los modelos de ML adecuadamente.

- __Datos recibidos:__ obtenidos de [el portal de calidad del aire de la CDMX](http://www.aire.cdmx.gob.mx/default.php?opc='aKBhnmI='&opcion=Zg==)
- __Responsable:__ Daniel Bustillos
- __Contacto:__  juandaniel.bucam@gmail.com

__Notas del proyecto__
Necesitamos generar un csv con los niveles máximos, mínimo y promedio por día, se perderá la información de la estación, también generaremos el atributo PM10mean y PM25mean que representarán los valores de las últimas 24 horas de los contaminantes.

calidad de caire
pronostico contaminación

## Pasos
[X] Leer los 10 archivos de contaminación <br>
[X] concatenar los archivos<br>
[X] Hacer el mes y el día columnas<br><br>
[X] Hacer el rollinng para PM10 y PM25<br><br>
[X] identidicar y eliminar Outliers
[X] Formatear Fechas
juntar<br>
[X] juntar archivos de append y rolling <br><br>


In [60]:
import numpy as np
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt

pd.options.mode.chained_assignment = None 

## Leamos los DF generados

Leemos los archivos por hora:

Declaramos todos los archivos enuna sola lista

In [44]:
lista = ["cont_hora2016.csv", "cont_hora2017.csv", "cont_hora2018.csv"
         ,"cont_hora2015.csv","cont_hora2019.csv"]

Leemos los archivos y creamos un diccionario:

In [45]:
data_hora = {}
for elem in lista:
    data_hora[elem[9:13]] = pd.read_csv(str('/home/paw/DanielBustillos/contaminación/datasets/por_hora/'+elem))
    data_hora[elem[9:13]] = data_hora[elem[9:13]].iloc[:,1:]

Apendeamos todos los DF

In [46]:
df_append = pd.DataFrame(columns=data_hora["2019"].columns.tolist())
for key in data_hora:
    df_append = df_append.append(data_hora[key], ignore_index=True,sort=True)

- Filtro fechas

Nos quedamos únicamente con los datos del 2010 en adelante:

In [47]:
df_append = df_append[df_append.fecha > "2016-01-01 23:00:00"]

Arreglamos las fechas, vamos a generar columnas con el atributo hora,dia,mes. Además, vamos a hacer que la columna fecha contenga la hora:

In [48]:
df_append['dia'] = df_append['fecha'].astype(str).str[8:10]
df_append['mes'] =  df_append['fecha'].astype(str).str[5:7]
df_append['fecha'] = df_append['fecha'].astype(str).str[0:10]

df_append['fecha']  = pd.to_datetime([''.join([' '.join([df_append.loc[i, 'fecha'], 
                                                         str(df_append.loc[i, 'hora'])]),':00']) for i in df_append.index])
df_append['fecha'] = pd.to_datetime(df_append['fecha'], format='%Y-%m-%d %H:%M:%S')

In [49]:
df_append.head()

,CO,NO,NO2,NOX,O3,PBa,PM10,PM2.5,PMCO,RH,...,TMP,UVA,UVB,WDR,WSP,dia,fecha,hora,id_station,mes
552,0.4,0.0,14.0,15.0,40.0,NaN,32.0,13.0,19.0,28.0,...,15.4,NaN,NaN,224.0,2.5,01,2016-02-01 01:00:00,1,AJM,02
553,NaN,NaN,NaN,NaN,5.0,NaN,NaN,15.0,NaN,86.0,...,2.6,NaN,NaN,210.0,1.7,01,2016-02-01 01:00:00,1,AJU,02
554,1.4,NaN,NaN,NaN,2.0,NaN,48.0,27.0,21.0,45.0,...,14.1,NaN,NaN,255.0,1.4,01,2016-02-01 01:00:00,1,BJU,02
555,1.1,19.0,41.0,60.0,1.0,NaN,43.0,20.0,23.0,NaN,...,NaN,NaN,NaN,NaN,NaN,01,2016-02-01 01:00:00,1,CAM,02
556,0.5,2.0,22.0,25.0,21.0,NaN,NaN,14.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,01,2016-02-01 01:00:00,1,CCA,02


Ordenemos el DF por fecha y estación:

In [50]:
df_append = df_append.sort_values(['id_station','fecha'], ascending=[1, 1])

In [51]:
#df_append = df_append.drop_duplicates()
#df_append = df_append[1::]
df_append = df_append.drop(["PBa","PMCO","UVA","UVB"],axis=1)

Vamos a quedarnos con las celdas donde Pm10 y Pm2.5 son no nulas.

In [52]:
df_append = df_append.reset_index(drop=True)#PM25
df_append["PM2.5"] = df_append["PM2.5"].astype(float)
df_append["NO"] = df_append["NO"].astype(float)

Grafiquemos PM10 y PM25:

Vemos que tenemos valores muy grandes, Jiang (2004) reportó que para valores mayores de 300 puntos, las mediciones no son confiables, vamos a quedarnos con mediciones menores a esta cota:

__Jiang, D., Zhang, Y., Hu, X., Zeng, Y., Tan, J., & Shao, D. (2004). Progress in developing an ANN model for air pollution index forecast. Atmospheric Environment, 38(40 SPEC.ISS.), 7055–7064. https://doi.org/10.1016/j.atmosenv.2003.10.066__

No queremos perder fechas, vamos a quedarnos con el valor anterior al outlier:


# Datos por día: mean(), max(), min()

### Creamos un Df con los datos promedio por hora oara rellenar los nan del DF original

In [53]:
df_append = df_append.sort_values(['fecha','id_station'], ascending=[1, 1])

In [54]:
df_append.head(5)

,CO,NO,NO2,NOX,O3,PM10,PM2.5,RH,SO2,TMP,WDR,WSP,dia,fecha,hora,id_station,mes
23978,0.6,2.0,24.0,26.0,14.0,NaN,NaN,62.0,2.0,14.8,189.0,1.9,02,2016-01-02,0,AJM,01
51941,NaN,NaN,NaN,NaN,16.0,NaN,3.0,92.0,NaN,8.5,195.0,1.3,02,2016-01-02,0,AJU,01
131383,1.1,15.0,40.0,55.0,2.0,53.0,17.0,NaN,19.0,NaN,NaN,NaN,02,2016-01-02,0,CAM,01
154797,0.9,3.0,39.0,42.0,11.0,NaN,19.0,NaN,NaN,NaN,NaN,NaN,02,2016-01-02,0,CCA,01
181618,0.6,NaN,NaN,NaN,27.0,21.0,NaN,65.0,1.0,14.0,149.0,1.0,02,2016-01-02,0,CHO,01


In [55]:
data_24 = df_append.set_index('fecha')

ahora vamos a sacar el promedio de las ultimas 24 horas por estación para el PM10 y el PM25:

In [56]:
data_24 = data_24.groupby('id_station')[["PM10","PM2.5"]].rolling(24).mean().dropna().reset_index()

In [57]:
data_24 = data_24.rename(columns={'PM10': 'PM10mean',"PM2.5":'PM25mean'})

## Unir los archivos:

Unimos los dos DF:

In [58]:
data_day = pd.merge(df_append, data_24, on=["fecha","id_station"],how="left")

In [59]:
data_day.to_csv(str("./datos_entrenamiento.csv"),index=False)